In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun
import numpy as np
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [6]:
numList=np.arange(1,50)
rdd=sc.parallelize(numList)
result=rdd.collect()
print(result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


Find the sum, average, maximum, minimum, and count

In [10]:
#Sum
rdd.sum()

1225

In [11]:
#avg
rdd.mean()

25.0

In [12]:
#Count
rdd.count()

49

In [13]:
#Min
rdd.min()

1

In [14]:
#Max
rdd.max()

49

Count how many numbers are even vs. odd.

In [17]:
even_count = rdd.filter(lambda x: x % 2 == 0).count()

# Count odds
odd_count = rdd.filter(lambda x: x % 2 != 0).count()

print(f"ODD Numbers: {odd_count}  EVEN Numbers: {even_count}")

ODD Numbers: 25  EVEN Numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [33]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
print(rdd_people.collect())

[('Nada ', 25), ('Mona', 30), ('Ahmed', 35), ('Khaled', 40), ('Ahmed', 35), ('Nada ', 25)]


Find the oldest person

In [23]:
rdd_people.max(key=lambda x: x[1])

('Khaled', 40)

Compute the average age

In [36]:
ages = rdd_people.map(lambda x: x[1])
average_age = ages.mean()
average_age

31.666666666666668

Group all the names by their age

In [37]:
age_keyed = rdd_people.map(lambda x: (x[1], x))

# Group by age
grouped = age_keyed.groupByKey().mapValues(list)

# Collect result
result = grouped.collect()
print(result)

[(40, [('Khaled', 40)]), (25, [('Nada ', 25), ('Nada ', 25)]), (35, [('Ahmed', 35), ('Ahmed', 35)]), (30, [('Mona', 30)])]


In [43]:
age_keyed = rdd_people.map(lambda x: (x[1], x))

# Group by age
grouped = age_keyed.groupByKey().mapValues(list)
grouped.collect()


[(40, [('Khaled', 40)]),
 (25, [('Nada ', 25), ('Nada ', 25)]),
 (35, [('Ahmed', 35), ('Ahmed', 35)]),
 (30, [('Mona', 30)])]

Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [47]:
rdd_russia = sc.textFile("/data/russia.txt")
rdd_russia.collect()

['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production']

Count the total number of lines.

In [48]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [ ]:
count_russia = rdd_russia.filter(lambda line: "Russia" in line).count()
count_russia

6

Find the most 5 frequent word in the file.

In [54]:
words = rdd_russia.flatMap(lambda line: line.lower().split())
word_counts = words.map(lambda w: (w, 1)) \
                   .reduceByKey(lambda x, y: x + y)
top5 = word_counts.takeOrdered(5, key=lambda x: -x[1])
print(top5)

[('the', 10), ('is', 7), ('russia', 5), ('in', 5), ('world', 3)]


Tokenize words

In [55]:
print(words.collect())

['russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'moscow', 'is', 'the', 'capital', 'city', 'of', 'russia', 'the', 'russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'the', 'trans-siberian', 'railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'the', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of).

In [56]:
stopwords = {"a", "the", "is", "to", "in", "of"}
filtered_words = words.filter(lambda w: w not in stopwords)
print(filtered_words.collect())

['russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'trans-siberian', 'railway', 'longest', 'railway', 'line', 'world', 'russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [57]:
filtered_counts = filtered_words.map(lambda w: (w, 1)) \
                                .reduceByKey(lambda x, y: x + y)

print(filtered_counts.collect())


[('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('capital', 1), ('russian', 1), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ('line', 1), ('literature,', 1), ('music', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('landscapes', 1), ('player', 1), ('energy', 1), ('production', 1), ('russia', 5), ('moscow', 1), ('city', 1), ('one', 1), ('spoken', 1), ('languages', 1), ('its', 2), ('rich', 1), ('culture', 1), ('trans-siberian', 1), ('railway', 2), ('longest', 1), ('has', 1), ('strong', 1), ('tradition', 1), ('ballet', 1), ('vast', 1), ('major', 1), ('global', 1)]


In [58]:
schema = 'id integer, name string, age integer, salary integer'
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [ ]:
# Show the schema
df.printSchema()

# Show first 2 rows
df.show(2)


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [62]:
df["name","salary"].show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [67]:
from pyspark.sql.functions import avg
df.select(avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [ ]:
df.filter(df.age > 28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [73]:
from pyspark.sql.functions import countDistinct

result = df.select(countDistinct("name")).collect()[0][0]
print(result)


4


Group by a the name column and find average salary

In [74]:
from pyspark.sql.functions import avg

df.groupBy("name").agg(avg("salary").alias("Agerage")).show()


+------+-------+
|  name|Agerage|
+------+-------+
|   Ali| 4000.0|
|Mariam| 6750.0|
|  Omar| 6750.0|
|  Sara| 5000.0|
+------+-------+



In [75]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Find the avg sales

In [76]:
avg_sales = df1.select(avg("sales")).collect()[0][0]
print(avg_sales)


400.5


Replace null name with 'Unknown' and sales with the avg sales of the column

In [77]:
from pyspark.sql.functions import when, col

df1_cleaned = df1.withColumn(
    "name",
    when(col("name").isNull(), "Unknown").otherwise(col("name"))
).withColumn(
    "sales",
    when(col("sales").isNull(), avg_sales).otherwise(col("sales"))
)

df1_cleaned.show()


+----+-------+-----+
|  Id|   name|sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

